In [23]:
!pip install more-itertools

  Using cached more_itertools-9.0.0-py3-none-any.whl (52 kB)


In [1]:
import pandas as pd
import nums_from_string

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import nums_from_string

ser = Service("./chromedriver")
options = Options()
options.headless = False
options.add_argument("--window-size=1920,1200")
def get_links():
    links = []
    driver = webdriver.Chrome(options=options, service=ser)
    driver.get("https://www.idealista.com/alquiler-viviendas/barcelona/eixample/l-antiga-esquerra-de-l-eixample/")
    for i in range(1): #cada uno son 25 links, antes era 8
        apts_links = driver.find_elements(By.CLASS_NAME, "item  item_contains_branding item-multimedia-container")
        for a in apts_links:
            links.append(a.find_element(By.CLASS_NAME, "item-link").get_attribute('href'))
        nb = driver.find_element(By.XPATH, "//a[contains(@class, 'icon-arrow-right-after')]")
        nb.click()
        time.sleep(2)
    df = pd.DataFrame(links)
    df = df.rename(columns={0: 'urls'})
    df.to_csv('urls_t.csv')
    driver.quit()
    print('DOOONEEEE')
    return

In [5]:
get_links()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[contains(@class, 'icon-arrow-right-after')]"}
  (Session info: chrome=109.0.5414.75)
Stacktrace:
Backtrace:
	(No symbol) [0x00406643]
	(No symbol) [0x0039BE21]
	(No symbol) [0x0029DA9D]
	(No symbol) [0x002D1342]
	(No symbol) [0x002D147B]
	(No symbol) [0x00308DC2]
	(No symbol) [0x002EFDC4]
	(No symbol) [0x00306B09]
	(No symbol) [0x002EFB76]
	(No symbol) [0x002C49C1]
	(No symbol) [0x002C5E5D]
	GetHandleVerifier [0x0067A142+2497106]
	GetHandleVerifier [0x006A85D3+2686691]
	GetHandleVerifier [0x006ABB9C+2700460]
	GetHandleVerifier [0x004B3B10+635936]
	(No symbol) [0x003A4A1F]
	(No symbol) [0x003AA418]
	(No symbol) [0x003AA505]
	(No symbol) [0x003B508B]
	BaseThreadInitThunk [0x765D00F9+25]
	RtlGetAppContainerNamedObjectPath [0x77377BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77377B8E+238]


In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import nums_from_string

ser = Service("./chromedriver")
options = Options()
options.headless = True
# options.add_argument("--window-size=1920,1200")
links = []
driver = webdriver.Chrome(options=options, service=ser)
driver.get("https://www.idealista.com/alquiler-viviendas/barcelona/eixample/l-antiga-esquerra-de-l-eixample/")

# apts_links = driver.find_elements(By.CLASS_NAME, "item  item_contains_branding item-multimedia-container")
apts_links = driver.find_elements(By.XPATH, "//div[contains(@class, 'listing-top')]")

print(apts_links)
print(type(apts_links))

[]
<class 'list'>


In [29]:
# !pip install selenium undetected-chromedriver

In [41]:
from selenium import webdriver
from selenium.webdriver.common.proxy import Proxy, ProxyType

options = webdriver.ChromeOptions()

proxy = Proxy()
proxy.proxyType = ProxyType.MANUAL
proxy.autodetect = False
proxy.httpProxy = proxy.sslProxy = proxy.socksProxy = "127.0.0.1:9000"
options.Proxy = proxy
options.add_argument("ignore-certificate-errors")


driver = webdriver.Chrome('./chromedriver', chrome_options=options) 
links = []
driver = webdriver.Chrome(options=options, service=ser)
driver.get("https://www.idealista.com/alquiler-viviendas/barcelona/eixample/l-antiga-esquerra-de-l-eixample/")

# apts_links = driver.find_elements(By.CLASS_NAME, "item  item_contains_branding item-multimedia-container")
apts_links = driver.find_elements(By.XPATH, "//div[contains(@class, 'listing-top')]")

print(apts_links)
print(type(apts_links))

C:\Users\ockda\AppData\Local\Temp\ipykernel_2084\3644287795.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver', chrome_options=options)
C:\Users\ockda\AppData\Local\Temp\ipykernel_2084\3644287795.py:14: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('./chromedriver', chrome_options=options)


[]
<class 'list'>


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

import time
import requests
from bs4 import BeautifulSoup

link = 'https://www.idealista.com/alquiler-viviendas/barcelona/eixample/l-antiga-esquerra-de-l-eixample/'

def filter_proxies():   
    response = requests.get('https://www.sslproxies.org/')
    soup = BeautifulSoup(response.text,"html.parser")
    proxies = []
    for item in soup.select("table.table tbody tr"):
        if not item.select_one("td"):break
        ip = item.select_one("td").text
        port = item.select_one("td:nth-of-type(2)").text
        proxies.append(f"{ip}:{port}")
    return proxies


def create_proxy_driver(PROXY):
    options = Options()
    options.add_argument("--headless")
    options.add_argument(f'--proxy-server={PROXY}')
    driver = webdriver.Chrome(options=options)
    return driver


def get_content(ALL_PROXIES,driver):
    while True:
        try:
            driver.get(link)
            title = driver.find_element(By.CSS_SELECTOR,"h3.s-post-summary--content-title > a").text
            if title:
                driver.quit()
                return title
        except Exception as e:
            driver.quit()
            if not ALL_PROXIES:
                print("Proxies used up (%s)" % len(ALL_PROXIES))
                ALL_PROXIES = filter_proxies()

            new_proxy = ALL_PROXIES.pop()
            driver = create_proxy_driver(new_proxy)
            ser = Service("./chromedriver")
            options = Options()
            options.headless = True
            options.add_argument("--window-size=1920,1200")
            links = []
            driver = webdriver.Chrome(options=options, service=ser)
            driver.get("https://www.idealista.com/alquiler-viviendas/barcelona/eixample/l-antiga-esquerra-de-l-eixample/")

            # apts_links = driver.find_elements(By.CLASS_NAME, "item  item_contains_branding item-multimedia-container")
            apts_links = driver.find_elements(By.XPATH, "//div[contains(@class, 'listing-top')]")

            print(apts_links)
            print(type(apts_links))
            print("proxy being used: %s" % new_proxy)
            time.sleep(1)


if __name__ == '__main__':
    ALL_PROXIES = filter_proxies()
    new_proxy = ALL_PROXIES.pop()
    driver = create_proxy_driver(new_proxy)
    
    
    print(get_content(ALL_PROXIES,driver))
    


In [20]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import os
import pandas as pd
%matplotlib inline
import requests
url = 'https://www.idealista.com/alquiler-viviendas/barcelona/eixample/l-antiga-esquerra-de-l-eixample/'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html')
# products = 

In [21]:
soup

<html><head><title>idealista.com</title><style>#cmsg{animation: A 1.5s;}@keyframes A{0%{opacity:0;}99%{opacity:0;}100%{opacity:1;}}</style></head><body style="margin:0"><p id="cmsg">Please enable JS and disable any ad blocker</p><script data-cfasync="false">var dd={'rt':'c','cid':'AHrlqAAAAAMANF0wiSCCqzUAAps23Q==','hsh':'AC81AADC3279CA4C7B968B717FBB30','t':'bv','s':17156,'e':'aa97e8cb273be2a04f1ad09e06b5cb6ad8822b5908ce31ae6e86cefa84df426e','host':'geo.captcha-delivery.com'}</script><script data-cfasync="false" src="https://ct.captcha-delivery.com/c.js"></script></body></html>